# Hoeffding Inequality

In [4]:
import random

In [2]:
# array represent the frequency of heads for each coin

sims = 100000
v_one_average = 0
v_rand = 0
v_min = 0
for n in range(sims):
  coins = [0] * 1000
  for j in range(len(coins)):
    total = 0
    for i in range(10):
      total += random.randint(0,1) / 10
      coins[j] = total
  v_one_average += coins[0]
  v_rand += coins[random.randint(0, 999)]
  v_min += min(coins)


print("v_one : " + str(v_one_average / sims))
print("v_rand : " + str(v_rand / sims))
print("v_min : " + str(v_min / sims))

v_one : 0.5002999999999991
v_rand : 0.5010170000000012
v_min : 0.037241999999977085


# Linear Regression

In [5]:
import numpy as np

In [6]:
# global vars
min = -1
max = 1
d = 2

def create_X(N):
  X = [(0, 0, 0) for i in range(N)]
  for i in range(N):
    X[i] = (random.uniform(min, max), random.uniform(min, max), 1) #(x, y, artifical bias)
  return np.array(X)

def get_target_function():
  x1 = random.uniform(min, max)
  y1 = random.uniform(min, max)
  x2 = random.uniform(min, max)
  y2 = random.uniform(min, max)
  m =  (y1 - y2) / (x1 - x2)
  b = y1 - (m * x1)
  return (m, b)

def create_y(X, target_func):
  y = []
  for i in range(X.shape[0]):
    point = X[i]
    point_class = -1
    if point[0] * target_func[0] + target_func[1] > point[1]:
      point_class = 1
    y.append(point_class)
  return np.array(y)

def find_weight_vec(X, y):
  pseudo_inv = np.dot(np.linalg.inv(np.dot(X.T, X)), X.T)
  w = np.array(np.dot(pseudo_inv, y))
  return w

def predict(w, point):
  guess = np.dot(w, point)
  if guess > 0: return 1
  else: return -1


def linear_regression(N):
  X = create_X(N)
  target_function = get_target_function()
  y = create_y(X, target_function)
  w = find_weight_vec(X, y)
  return w, target_function, X, y

def in_sample_points_eval(N):
  incorrect = 0
  w, target, X, y = linear_regression(N)
  for i in range(X.shape[0]):
    point = X[i]
    if predict(w, point) != y[i]:
      incorrect += 1
  return incorrect / N

def out_of_sample_points_eval(N):
  incorrect = 0
  w, target, X, y = linear_regression(N)
  num_points = 1000
  new_points = create_X(num_points)
  new_point_classification = create_y(new_points, target)
  for i in range(new_points.shape[0]):
    point = new_points[i]
    if predict(w, point) != new_point_classification[i]:
      incorrect += 1
  return incorrect / num_points


In [7]:
sims = 1000
total = 0
for i in range(sims):
  total += in_sample_points_eval(100)
print("The average fraction of incorrectly classified points is: " + str(total / sims) + " for in sample")

The average fraction of incorrectly classified points is: 0.04035000000000011 for in sample


In [8]:
sims = 1000
total = 0
for i in range(sims):
  total += out_of_sample_points_eval(100)
print("The average fraction of incorrectly classified points is: " + str(total / sims) + " for out of sample")


The average fraction of incorrectly classified points is: 0.047124000000000006 for out of sample


In [9]:
def create_points(N):
  X = [(0, 0, 0) for i in range(N)]
  for i in range(N):
    X[i] = (random.uniform(min, max), random.uniform(min, max), 1) #(x, y, artifical bias)
  return np.array(X)

def get_target_function():
  x1 = random.uniform(min, max)
  y1 = random.uniform(min, max)
  x2 = random.uniform(min, max)
  y2 = random.uniform(min, max)
  m =  (y1 - y2) / (x1 - x2)
  b = y1 - (m * x1)
  return (m, b)

def create_y(X, target_func):
  y = []
  for i in range(X.shape[0]):
    point = X[i]
    point_class = -1
    if point[0] * target_func[0] + target_func[1] > point[1]:
      point_class = 1
    y.append(point_class)
  return np.array(y)

def update_weights(w, point, point_class):
  w[0] += point[0] * point_class
  w[1] += point[1] * point_class
  w[2] += point[2] * point_class

def guess_class(w, point):
  predict =  np.dot(w, point)
  if predict > 0: return 1
  else: return -1

def PLA(N, target_func, w):
  count = 0
  X = create_points(N)
  y = create_y(X, target_func)
  i = 0
  while i < X.shape[0]:
    point = X[i]
    point_class = y[i]
    guess = guess_class(w, point)
    if guess != point_class:
      update_weights(w, point, point_class)
      i = 0
      count += 1
    else:
      i += 1
  return w, count

def convergence_time(N):
  sims = 1000
  total = 0
  for i in range(sims):
    w, target, X, y = linear_regression(N)
    _, step_count = PLA(N, target, w)
    total += step_count
  print("The average convergence is " + str(total / sims) + " for N = " + str(N))

In [10]:
convergence_time(10)

The average convergence is 8.627 for N = 10


#Non-Linear Transformation

In [11]:
"""
rest of functions used in linear regression defined earlier
"""
def create_noisy_y(X):
  y = []
  for i in range(X.shape[0]):
    point = X[i]
    point_class = np.sign(point[0]**2 + point[1]**2 - 0.6)
    y.append(point_class)
  # pick random indices to flip with no replacement
  indices = np.random.choice(X.shape[0], X.shape[0] // 10, replace=False)
  for idx in indices:
    y[idx] = -1 * y[idx]
  return np.array(y)

def predict(w, point):
  return np.sign(np.dot(w, point))

def noisy_linear_regression(N):
  X = create_X(N)
  #f =  np.sign(point[0]**2 + point[1]**2 - 0.6)
  y = create_noisy_y(X)
  w = find_weight_vec(X, y)
  return w, X, y


def noisy_in_sample_points_eval(N):
  incorrect = 0
  w, X, y = noisy_linear_regression(N)
  for i in range(X.shape[0]):
    point = X[i]
    if predict(w, point) != y[i]:
      incorrect += 1
  return incorrect / N


In [12]:
# now simulate flipping some subset with noise to be incorrect
sims = 1000
total = 0
for i in range(sims):
  total += noisy_in_sample_points_eval(1000)
print("The average fraction of incorrectly classified points, E_in for a noisy sample of f is: " + str(total / sims) + " for N = 1000")

The average fraction of incorrectly classified points, E_in for a noisy sample of f is: 0.5055090000000001 for N = 1000


In [13]:
def create_transformed_X(N):
  X = [(0, 0, 0, 0, 0, 0) for i in range(N)]
  for i in range(N):
    x1 = random.uniform(min, max)
    x2 = random.uniform(min, max)
    X[i] = (1, x1, x2, x1 * x2, x1**2, x2**2)
    #(changing the order of a point compared to earlier)
  return np.array(X)

def create_transformed_noisy_y(X):
  y = []
  for i in range(X.shape[0]):
    point = X[i]
    point_class = np.sign(point[1]**2 + point[2]**2 - 0.6)
    y.append(point_class)
  indices = np.random.choice(X.shape[0], X.shape[0] // 10, replace=False)
  for idx in indices:
    y[idx] = -1 * y[idx]
  return np.array(y)

def transformed_noisy_linear_regression(N):
  X = create_transformed_X(N)
  #f =  np.sign(point[0]**2 + point[1]**2 - 0.6)
  y = create_transformed_noisy_y(X)
  w = find_weight_vec(X, y)
  return w, X, y

In [14]:
N = 1000
sims = 1000
w_avg = [0] * 6
for i in range(sims):
  w_new, _, _ = transformed_noisy_linear_regression(N)
  for j in range(len(w_avg)):
    w_avg[j] += w_new[j]
for j in range(len(w_avg)):
    w_avg[j] /= sims
print(str(w_avg))

[-0.9932198385686883, 0.0013060272363378596, -0.00043466277014730844, -0.0015980055310050659, 1.5584517271093243, 1.5623123732805801]


In [16]:
# for number 10
def out_of_sample_transformed_random_points_eval(N):
  incorrect = 0
  w, X, y = transformed_noisy_linear_regression(N)
  num_points = 1000
  new_points = create_transformed_X(num_points)
  new_point_classification = create_transformed_noisy_y(new_points)
  for i in range(new_points.shape[0]):
    point = new_points[i]
    if predict(w, point) != new_point_classification[i]:
      incorrect += 1
  return incorrect / num_points


In [17]:
sims = 1000
total = 0
for i in range(sims):
  total += out_of_sample_transformed_random_points_eval(1000)
print("The average fraction of incorrectly classified points is: " + str(total / sims) + " for out of sample with a nonlinear f")


The average fraction of incorrectly classified points is: 0.12622599999999998 for out of sample with a nonlinear f
